In [10]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.wait import WebDriverWait
import time
import re
from tqdm import tqdm
import pandas as pd


In [91]:
class Collector:
    companyNameList = []
    workExperienceList = []
    educationList = []
    addressList = []
    employeeNumberList = []
    yearsList = []
    typeList = []
    skillList = []

    def __init__(self):
        self.driver = webdriver.Chrome(service=Service("../driver/chromedriver"))
        
    def getStart(self):
        self.driver.get("https://www.jobkorea.co.kr/Search/?stext=%EB%A1%9C%EB%B4%87%20%EC%86%8C%ED%94%84%ED%8A%B8%EC%9B%A8%EC%96%B4&tabType=recruit&Page_No=1")
        
    def executeCollect(self):
        self.companyNameList = []
        self.workExperienceList = []
        self.educationList = []
        self.addressList = []
        self.employeeNumberList = []
        self.yearsList = []
        self.typeList = []
        self.skillList = []
        while True:
            time.sleep(2)
            currentPage = Page(self)
            currentPage.executeCollect()
            print(len(self.companyNameList), len(self.workExperienceList), len(self.addressList), len(self.employeeNumberList), len(self.typeList))
            try:
                self.driver.find_element(By.CLASS_NAME, "button-next").click()
            except:
                break
            


class Page:
    def __init__(self, collector:Collector):
        self.co = collector
        self.employmentList = self.co.driver.find_element(By.CLASS_NAME, "list").find_elements(By.CLASS_NAME, "list-item")
        

    def getPageInfo(self, employment):
        infoList = employment.find_element(By.CLASS_NAME, "chip-information-group").find_elements(By.TAG_NAME, "li")
        self.co.workExperienceList.append(infoList[0].text)
        self.co.educationList.append(infoList[1].text)
        self.co.addressList.append(infoList[3].text)
        link = employment.find_element(By.CLASS_NAME, "corp-name-link.dev-view")
        self.co.companyNameList.append(link.text)
        print(infoList[0].text, infoList[1].text, infoList[3].text, link.text)
        
    def clickEmployment(self, employment):
        employment.find_element(By.CLASS_NAME, "corp-name-link.dev-view").click()

    def executeCollect(self):
        for employment in tqdm(self.employmentList, total=len(self.employmentList)):
            self.getPageInfo(employment)
            self.clickEmployment(employment)

            currentEmployment = Employment(self.co)
            currentEmployment.executeCollect()

        
class Employment:
    def __init__(self, collector:Collector):
        self.co = collector
        self.originalHandle = self.co.driver.current_window_handle
        self.loading()
        self.soup = self.getSoup()

    def loading(self):
        WebDriverWait(self.co.driver, 10).until(EC.number_of_windows_to_be(2))
        for handle in self.co.driver.window_handles:
            if handle != self.originalHandle:
                self.co.driver.switch_to.window(handle)
        self.co.driver.implicitly_wait(10)

    def getSoup(self):
        page = self.co.driver.page_source
        return BeautifulSoup(page, "html.parser")
        
    def executeCollect(self):
        self.getEmploymentInfo(self.soup)
        self.co.driver.close()
        self.co.driver.switch_to.window(self.originalHandle)

    def getEmploymentInfo(self, soup):
        try:
            infoList = soup.find(class_="tbCol coInfo").find_all("dd")
        except:
            self.co.employeeNumberList.append(None)
            self.co.yearsList.append(None)
            self.co.typeList.append(None)
            return -1

        employee = None
        years = None
        companyType = None

        for each in infoList:
            if re.search("\d+명", each.getText()):
                employee = re.search("\d+명", each.getText()).group()[:-1]
            if re.search("\w+업", each.getText()):
                companyType = re.search("\w+업", each.getText()).group()
            if re.search("\d+년차", each.getText()):
                years = re.search("\d+년차", each.getText()).group()[:-2]
        self.co.employeeNumberList.append(employee)
        self.co.yearsList.append(years)
        self.co.typeList.append(companyType)
        print(employee, years, companyType)
        

In [85]:
jobKoreaCollector = Collector()
jobKoreaCollector.getStart()

In [87]:
jobKoreaCollector.executeCollect()

  0%|          | 0/20 [00:00<?, ?it/s]

경력3년↑ 대졸↑ 경기 성남시 에코트로닉스㈜


  5%|▌         | 1/20 [00:02<00:44,  2.36s/it]

90 34 중소기업
경력2년↑ 초대졸↑ 경기 성남시 로보케어


 10%|█         | 2/20 [00:05<00:46,  2.56s/it]

...

경력5년↑ 대졸↑ 서울 강남구 외 대기업 계열사·자회사


 86%|████████▌ | 6/7 [00:10<00:01,  1.78s/it]

경력13년↑ 대졸↑ 대구 달성군 잡뉴스솔로몬서치


100%|██████████| 7/7 [00:11<00:00,  1.71s/it]


227 227 227 227 227


In [90]:
company = {"기업명":jobKoreaCollector.companyNameList,
           "설립일":jobKoreaCollector.yearsList,
           "기업형태":jobKoreaCollector.typeList,
           "사원수":jobKoreaCollector.employeeNumberList,
           "주소":jobKoreaCollector.addressList,
           }
employment = {
    "회사명":jobKoreaCollector.companyNameList,
    "기술":None,
    "경력":jobKoreaCollector.workExperienceList,
    "학력":jobKoreaCollector.educationList
}

company = pd.DataFrame(company)
employment = pd.DataFrame(employment)
company.to_csv("../data/company.csv")
employment.to_csv("../data/employment.csv")


company

,기업명,설립일,기업형태,사원수,주소
0,에코트로닉스㈜,34,중소기업,90,경기 성남시
1,로보케어,13,중견기업,43,경기 성남시
2,㈜라이온로보틱스,2,중소기업,7,대전 유성구
3,㈜RGA Inc.,3,벤처기업,4,서울 서초구
4,아이아이컴바인드,14,중견기업,825,서울 마포구
...,...,...,...,...,...
222,중소기업(300명이하),None,None,None,경기 오산시 외
223,중견기업(300명이상),None,None,None,대구 달성군
224,중견기업(300명이상),None,None,None,대구 달성군
225,대기업 계열사·자회사,None,None,None,서울 강남구 외
